# Logistic Regression

## Load digits dataset

In [1]:
from sklearn import datasets

In [2]:
digits = datasets.load_digits()
data = digits.data
labels = digits.target
data.shape, labels.shape

((1797, 64), (1797,))

## Convert labels into one-hot vectors

In [3]:
from sklearn.preprocessing import OneHotEncoder

In [4]:
one_hot = OneHotEncoder(sparse=False, dtype=int)
labels = labels.reshape(-1, 1) # convert (n,) -> (n,1)
labels = one_hot.fit_transform(labels)

## Split into training and test sets

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

## Build the model

In [7]:
import tensorflow as tf

create variables and input nodes

In [8]:
x_dim = X_train.shape[1]
y_dim = y_train.shape[1]

W = tf.Variable(tf.random_normal([x_dim, y_dim], stddev=0.01), name='weights')
b = tf.Variable(tf.zeros([y_dim]), name="biases")

X = tf.placeholder(tf.float32, [None, x_dim], name='X')
y = tf.placeholder(tf.float32, [None, y_dim], name='y')

wire the model

In [9]:
logits = tf.matmul(X, W) + b
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, y, name='xentropy')
loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')

create optimizer

In [10]:
tf.scalar_summary(loss.op.name, loss)
lr = 0.05 # learning rate
optimizer = tf.train.GradientDescentOptimizer(lr)
# Create a variable to track the global step.
global_step = tf.Variable(0, name='global_step', trainable=False)
# Use the optimizer to apply the gradients that minimize the loss
# (and also increment the global step counter) as a single training step.
train_op = optimizer.minimize(loss, global_step=global_step)

create evaluation and prediction subgraph

In [11]:
correct = tf.equal(tf.argmax(y,1), tf.argmax(logits,1))
eval_op = tf.reduce_mean(tf.cast(correct, tf.float32))
prediction_op = tf.argmax(logits, 1)

In [13]:
init_op = tf.initialize_all_variables()

In [14]:
batch_size = 100
n_epochs = 20
with tf.Session() as sess:
    sess.run(init_op) # init all variables
    
    for e in range(n_epochs):
        for start in range(0, len(X_train), batch_size):
            end = start + batch_size
            batch = {X: X_train[start:end], y: y_train[start:end]}

            _, loss_val = sess.run([train_op, loss], feed_dict=batch)
        
        eval_val = sess.run([eval_op], feed_dict={X: X_test, y: y_test})
        print('accuracy: ', eval_val[0])

accuracy:  0.847222
accuracy:  0.913889
accuracy:  0.925
accuracy:  0.927778
accuracy:  0.936111
accuracy:  0.955556
accuracy:  0.941667
accuracy:  0.95
accuracy:  0.952778
accuracy:  0.961111
accuracy:  0.963889
accuracy:  0.963889
accuracy:  0.963889
accuracy:  0.972222
accuracy:  0.972222
accuracy:  0.972222
accuracy:  0.969444
accuracy:  0.972222
accuracy:  0.972222
accuracy:  0.972222
